# Introductory quantum query algorithm implementations in Cirq.

In [1]:
# Import packages
import cirq
import numpy as np
import qiskit



## Deutsch Problem

Input : A function $f : \{0,1\} \rightarrow \{0,1\}$.

Output : Whether the function is constant or balanced. 0 if the function is constant, 1 if balanced.

Consider Boolean functions with one bit input $f_{i}(x)$. Such functions are either __constant__ or __balanced__. Here, $i = 0, 1$ are constant functions : $f_0(x) = 0$, and $f_1(x) = 1$, and $i = x, \bar{x}$ are balanced functions : $f_x(x) = x$ and $f_{\bar{x}}(x) = \bar{x}$.

 Classically, to differentiate between balanced and constant functions, one needs to query the function twice, both for $x = 0$ and $x = 1$. However, when the problem is framed in terms of qubits and the action of the function in terms of a unitary on the qubits, then constant or balanced functions can be differentiated using just one query.

The unitaries are defined according to the rule : $U_{i} : \ket{x, y} \rightarrow \ket{x, y + f_i(x)}$.

This is done by the following circuit :

In [5]:
# Define two qubits.
q0, q1 = cirq.LineQubit.range(2)

# The four kinds of Boolean functions on a single qubit, written as unitary gates on two qubits.
oracles = {'0':[], 
           '1':[cirq.X(q1)],
         'x':[cirq.CNOT(q0,q1)],
          'xbar' : [cirq.CNOT(q0,q1), cirq.X(q1)]
         }

# The circuit that differentiates between constant and balanced functions.
def deutsch(oracle):
    yield cirq.H(q0), cirq.X(q1)
    yield cirq.H(q1)
    yield oracle
    yield cirq.H(q0)
    yield cirq.measure(q0)

# Displaying the circuit for different choices of the oracle.
for key, oracle in oracles.items():
    print(f"For function with i : {key} \n ")
    print(cirq.Circuit(deutsch(oracle)))
    


For function with i : 0 
 
0: ───H───H───M───

1: ───X───H───────
For function with i : 1 
 
0: ───H───H───M───

1: ───X───H───X───
For function with i : x 
 
0: ───H───────@───H───M───
              │
1: ───X───H───X───────────
For function with i : xbar 
 
0: ───H───────@───H───M───
              │
1: ───X───H───X───X───────


In [6]:
# When measuring the first qubit q_0, we find constant functions always give 0 as output while balanced functions give 1.
for key, oracle in oracles.items():
    simulator = cirq.Simulator()
    result = simulator.run(cirq.Circuit(deutsch(oracle)), repetitions = 10)
    print(f"For key {key}\n")
    print(result)


For key 0

q(0)=0000000000
For key 1

q(0)=0000000000
For key x

q(0)=1111111111
For key xbar

q(0)=1111111111


These same circuits can also be drawn in Qiskit.

In [7]:
# Circuit in QISKIT.
from qiskit import QuantumCircuit

def deutsch_function(oracle):
    f = QuantumCircuit(2,1)
    f.h(0)
    f.x(1)
    f.h(1)
    if oracle in ["x", "xbar"]:
        f.cx(0,1)
    if oracle in ["xbar", "1"]:
        f.x(1)
    f.h(0)
    f.measure(0,0)
    return f


In [8]:
for i in ["0", "x", "xbar", "1"]:
    print(i)
    display(deutsch_function(i).draw())

0


┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ H ├┤M├
     ├───┤├───┤└╥┘
q_1: ┤ X ├┤ H ├─╫─
     └───┘└───┘ ║ 
c: 1/═══════════╩═
                0

x


┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐└───┘└╥┘
q_1: ┤ X ├┤ H ├┤ X ├──────╫─
     └───┘└───┘└───┘      ║ 
c: 1/═════════════════════╩═
                          0

xbar


┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐├───┤└╥┘
q_1: ┤ X ├┤ H ├┤ X ├┤ X ├─╫─
     └───┘└───┘└───┘└───┘ ║ 
c: 1/═════════════════════╩═
                          0

1


┌───┐┌───┐     ┌─┐
q_0: ┤ H ├┤ H ├─────┤M├
     ├───┤├───┤┌───┐└╥┘
q_1: ┤ X ├┤ H ├┤ X ├─╫─
     └───┘└───┘└───┘ ║ 
c: 1/════════════════╩═
                     0

## Duetsch - Jozsa problem

Input : A function $f : \{0,1\}^n \rightarrow \{0,1\}$

Promise : The function is either constant or balanced.

Output : $\ket{0}^{\otimes n}$ if $f$ is constant, some other state if balanced.


The Deutsch problem has an n-qubit generalisation. Given a Boolean function on n-qubits and an assurance that the oracle is either balanced or constant, we can again differentiate between the two using just one query and a similar circuit.

Classically, this would take $2^{n-1} + 1$ queries. However, a classical approach that is probabilistic can do much better. After $k$ queries, the probability that a balanced function would be mislabelled as constant drops exponentially $2^{-(k-1)}$.

Consider the case $n = 2$:


In [9]:
q0, q1, q2 = cirq.LineQubit.range(3)

oracles_constant = {'c_0':[],
                   'c_1':[cirq.X(q2)]}
oracles_balanced = {'1':[cirq.X(q0), cirq.CNOT(q0,q2), cirq.X(q0)],
                   '2' : [cirq.X(q1), cirq.CNOT(q1,q2), cirq.X(q1)],
                   '3' : [cirq.X(q0), cirq.X(q1), cirq.CNOT(q0,q2), cirq.CNOT(q1,q2), cirq.X(q0), cirq.X(q1)],
                    '1bar' : [cirq.CNOT(q0,q2)],
                    '2bar' : [cirq.CNOT(q1,q2)],
                    '3bar' : [cirq.CNOT(q0,q2), cirq.CNOT(q1,q2)]
                   }
def deutsch_josza_n2(oracle):
    yield cirq.H(q0), cirq.H(q1)
    yield cirq.X(q2), cirq.H(q2)
    yield oracle
    yield cirq.H(q0), cirq.H(q1)




print("Constant functions \n")
for key, oracle in oracles_constant.items():
    circuit = cirq.Circuit(deutsch_josza_n2(oracle))
    circuit.append(cirq.measure(q0, q1), strategy = cirq.InsertStrategy.NEW)
    print(f"Function key : {key}\n")
    print(circuit)
    simulator = cirq.Simulator()
    results = simulator.run(circuit, repetitions = 10)
    print(results,"\n")

print("Balanced functions \n")
for key, oracle in oracles_balanced.items():
    circuit = cirq.Circuit(deutsch_josza_n2(oracle))
    circuit.append(cirq.measure(q0, q1), strategy = cirq.InsertStrategy.NEW)
    print(f"Function key : {key}\n")
    print(circuit)
    simulator = cirq.Simulator()
    results = simulator.run(circuit, repetitions = 10)
    print(results,"\n")


Constant functions 

Function key : c_0

0: ───H───H───M───
              │
1: ───H───H───M───

2: ───X───H───────
q(0),q(1)=0000000000, 0000000000 

Function key : c_1

0: ───H───H───────M───
                  │
1: ───H───H───────M───

2: ───X───H───X───────
q(0),q(1)=0000000000, 0000000000 

Balanced functions 

Function key : 1

0: ───H───X───@───X───H───M───
              │           │
1: ───H───H───┼───────────M───
              │
2: ───X───H───X───────────────
q(0),q(1)=1111111111, 0000000000 

Function key : 2

0: ───H───H───────────────M───
                          │
1: ───H───X───@───X───H───M───
              │
2: ───X───H───X───────────────
q(0),q(1)=0000000000, 1111111111 

Function key : 3

0: ───H───X───@───X───H───────M───
              │               │
1: ───H───X───┼───@───X───H───M───
              │   │
2: ───X───H───X───X───────────────
q(0),q(1)=1111111111, 1111111111 

Function key : 1bar

0: ───H───────@───H───M───
              │       │
1: ───H───H───┼───────

We find no support for the $\ket{00}$ state in the case of balanced functions while constant functions only output $\ket{00}$.

## Bernstein-Vazirani Problem

Input : A function $f : \{0,1\}^n \rightarrow \{0,1\}$.

Promise : There exists binary sequence $a$ so that $f(x) = a.x$.

Output : The string a.

 The Deutsch - Josza circuit can also be used to find the function $f(x) = a.x$ for some n-bit string $a$. In this case, the output is the state $\ket{a}$ when the input is $\ket{0, 0, ...., 0}$. Classically, we can find the string in n queries, evaluating each bit in $a$ one by one. The circuit finds the sequence in one query.

Example for 5 qubits :


In [10]:
q0, q1, q2, q3, q4, q5 = cirq.LineQubit.range(6)
a = [0, 1, 0, 1, 1]
print("a =", a)


simulator = cirq.Simulator()
def bv_algorithm():
    yield cirq.X(q5)
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2), cirq.H(q3), cirq.H(q4), cirq.H(q5)
    yield cirq.CNOT(q1,q5), cirq.CNOT(q3,q5), cirq.CNOT(q4,q5) # the control qubits are the non-zero elements in a.
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2), cirq.H(q3), cirq.H(q4)
    yield cirq.measure(q0,q1,q2,q3,q4)

results = simulator.run(cirq.Circuit(bv_algorithm()))
print("\nThe qubits q0,q1,q2,q3 and q4 have states :\n")
print(results)


a = [0, 1, 0, 1, 1]

The qubits q0,q1,q2,q3 and q4 have states :

q(0),q(1),q(2),q(3),q(4)=0, 1, 0, 1, 1


To derive the above result formally, note that $H^{\otimes n} \ket{x} \sim \sum_y (-1)^{x.y} \ket{y}$.

### Simon's problem

Input : Two-to-one functions $f : \{0,1\}^n \rightarrow \{0,1\}^m$.

Promise : There is a period string $a$ such that $f(x) = f(x + a)$ for all x.

Output : The string $a$.

 Consider 2-to-1 functions on n-qubits with an m-qubit output, which are periodic with n-bit period $a$. Then the problem of finding the period of the function is known as Simon's problem. The circuit can be summarized as follows :

First we apply the Hadamard operators on the n-qubit register :
$\ket{0^{\otimes n}, 0^{\otimes m}} \rightarrow  \sum_{x = 0}^{2^n - 1} \ket{x} \ket{0^{\otimes m}}$.

Then we apply the unitary corresponding to the periodic function:
 $\sum_{x = 0}^{2^n - 1} \ket{x} \ket{0^{\otimes m}} \rightarrow \sum_{x = 0}^{2^n - 1} \ket{x} \ket{f(x)}$.

Then we make measurements on the second register of qubits. Say we get some state $\ket{f(x_0)}$ after the measurement, then the state in the first register would be $\sim \ket{x_0} + \ket{x_0 + a}$. Applying Hadamard transformations on this first register and then measuring the state gives possible output states $\ket{y}$ which satisfy the condition $y.a = 0$. From this information, we can obtain $a$ by repeatedly observing $\ket{y}$.


As an example, consider the periodic function on two qubits with the mapping :

$f : (x_0,x_1) \rightarrow (x_1,x_1)$ so that

$(0,0) \rightarrow (0,0)$

$(0,1) \rightarrow (1,1)$

$(1,0) \rightarrow (0,0)$

$(1,1) \rightarrow (1,1)$

Here, the period is $a = (1, 0)$.


In [11]:
# Since we need an output with two registers, we keep 4 qubits.

q0, q1, q2, q3 = cirq.LineQubit.range(4)

def simon_algorithm():
    yield cirq.H(q0), cirq.H(q1)
    yield cirq.CNOT(q1,q2), cirq.CNOT(q1,q3)
    yield cirq.measure(q2,q3)
    yield cirq.H(q0), cirq.H(q1)
    yield cirq.measure(q0,q1)

simulator = cirq.Simulator()
results = simulator.run(cirq.Circuit(simon_algorithm()), repetitions = 10)
print(cirq.Circuit(simon_algorithm()))
print(results)



0: ───H───H───────────M───
                      │
1: ───H───@───@───H───M───
          │   │
2: ───────X───┼───M───────
              │   │
3: ───────────X───M───────
q(0),q(1)=0000000000, 0111000101
q(2),q(3)=1001101000, 1001101000


Unique $q_0, q_1$ values are : (0,0), (0,1). Since $y.a = 0$, we find that $a = (a_0,0)$. Since the period cannot be $(0,0)$, it has to be $(1,0)$ which is the correct value for $a$. After n queries, the probability of failure of finding $a$ in this procedure decreases exponentially. On the other hand, finding the period is a very hard problem classically. Once, we evaluate the query and get some output $f(x)$, the only way to get information about the period is to evaluate the function again and get the same value f(x) for a different input. But $x+a$ is only one in an exponential number of possible inputs making a plausible solution hard.